In [ ]:
from mitmproxy import http, ctx
import time, re
import logging
import os

In [ ]:
distr = str(input("Directory: ")) #Директория для файлов
start_file = distr + str(input("File to save filtred stream to: ")) #Файл, в который пойдёт весь отфильтрованый поток
data_file = distr + str(input("Final file: ")) #Файл, куда будут сохранены нужные, раскодированные данные
script = "mitmdump_script.py" #Скрипт, при помощи которого достаём необходимые данные
with open ("save.txt", "w") as f: #Файл для передачи названия конечного файла скрипту
    f.write(data_file)

In [ ]:
def StartMITMDump(file):
    os.system("/home/michael/.local/bin/mitmdump -w " + file + " '~t video/webm | ~t video/mp4' &")
def StopMITMDump():
    time.sleep(int(input("Time to wait before stopping the flow: ")))
    os.system("pkill mitmdump")
def ReceiveData(script, file):
    os.system("/home/michael/.local/bin/mitmdump   -s " + script + " -nr " + file)

In [ ]:
StartMITMDump(start_file) #Запускаем mitmdump (не включать mitmproxy, не использовать прокси сервер для локалки). 
# Собираем потоки в файл.

In [ ]:
StopMITMDump() #Указываем через какое время в секундах остановить процесс mitmdump. Можно взять время с запасом
# (так как ещё есть реклама) и заниматься другими делами, предварительно отключив автовоспроизведение на ютубе.

In [ ]:
ReceiveData(script, start_file) #Записываем нужные данные в файл при помощи скрипта.

In [ ]:
d = {}
# https://gist.github.com/sidneys/7095afe4da4ae58694d128b1034e01e2
mapp = {"133":"240 px", "134":"360 px", "135": "480 px", "136": "720 px", "137": "1080 px", "160": "144 px", 
       "264": "1440 px", "298": "720 px", "299": "1080 px", "167": "360 px", "168": "480 px", "169": "720 px",
       "170" : "1080 px", "218": "480 px", "219": "144 px", "242": "240 px", "243": "360 px", "244": "480 px", 
       "245": "480 px", "246": "480 px", "247": "720 px", "248": "1080 px", "271": "1440 px", "278": "144 px",
       "298": "720 px", "299": "1080 px", "302": "720 px", "303": "1080 px", "308": "1440 px", "313": "2160 px", 
        "315": "2160 px", "330": "144 px", "331": "240 px", "332": "360 px", "333": "480 px", "334": "720 px", 
        "335": "1080 px", "336": "1440 px", "337": "2160 px", "272": "4320 px", "394": "144 px", "395": "240 px", 
        "396": "360 px", "397": "480 px", "398": "720 px", "399": "1080 px", "400": "1440 px", "401": "2160 px", 
        "402": "2880 px"} 
#Нужно убедиться в правильности этого, мог запутаться.
with open(data_file, "r") as f:
    a = [line.replace("\n","").split(",") for line in f.readlines()]
for idx, val in enumerate(a):
    d[idx] = {"mime":val[0], "aitag":mapp[val[1]], "duration":val[2], "range":val[3], "buffer":val[4], "clen":val[5],
             "first request byte":val[6], "request complete":val[7], "first response byte":val[8],
             "response complete":val[9]}
d #Словарь для удобства

In [ ]:
conn = sqlite3.connect(database) 
cursor = conn.cursor()

for row in a:
    row[1] = mapp[row[1]]
db_rows = [tuple(val) for val in a]
    
try:
    cursor.execute("""CREATE TABLE request_data
                  (mime text, aitag text, duration text,
                   range text, buffer text, first_request_byte text, request_complete text, 
                   first_response_byte text, response_complete text) """)
except:
    pass
 
cursor.executemany("INSERT INTO request_data VALUES (?,?,?,?,?,?,?,?,?)", db_rows)
conn.commit()

cursor.execute("SELECT * FROM request_data")
for idx, row in enumerate(cursor.fetchall()):
    print(str(idx) + ".", row)
cursor.close()
conn.close()